In [1]:
!df ~ --block-size=G

Filesystem     1G-blocks  Used Available Use% Mounted on
overlay             359G    8G      333G   3% /


In [2]:
%%sh
# at /content/two-stream-action-recognition
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_jpegs_256.zip.001 &
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_jpegs_256.zip.002 &
wget http://ftp.tugraz.at/pub/feichtenhofer/tsfusion/data/ucf101_jpegs_256.zip.003 &

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:

%%sh
# at /content/two-stream-action-recognition
cat ucf101_jpegs_256.zip.00* > ucf101_jpegs_256.zip &


In [0]:
# at /content/two-stream-action-recognition
!rm ucf101_jpegs_256.zip.00*

In [0]:
%%sh
# at /content/two-stream-action-recognition
unzip ucf101_jpegs_256.zip >> zip1.out &


In [0]:
!rm ucf101_jpegs_256.zip

In [7]:
!du -sh --human-readable *

4.0K	classInd.txt
33G	jpegs_256
55M	sample_data
144K	testlist01.txt
388K	trainlist01.txt
149M	zip1.out


In [30]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

    100% |████████████████████████████████| 592.3MB 47.9MB/s 


In [31]:
!pip3 install torchvision

    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 2.0MB 14.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch.utils.data as data

from PIL import Image
import os
import os.path
import numpy as np
from numpy.random import randint

In [0]:
class VidInfo(object):
  '''
  row: Vidoes info imported form file list [Directory,# of frames,label]
  '''
    def __init__(self, row):
        self.data = row

    @property  
    #property: Insted of writing obj.path(), you can write obj.path .It conceals the intial value of row/data inside the fuction and can't be changed 
    #even if data is overwrited later
    def path(self):
        return self.data[0]

    @property
    def num_frames(self):
        return int(self.data[1])

    @property
    def label(self):
        return int(self.data[2])

In [0]:
class TSNdataset(data.Dataset):
  '''
  file_list         : FileList.txt 
  num_segments      : Default = 3
  new_length        : The num of sequentially picked from the frames (1 'RGB' , 2 with 'RGBDiff')
  modality          : Takes two values 'RGB'&'RGBDiff'
  image_prefix      : Prefix of the frames name
  transform         : Pytorch composion of transformations (Data Augmentation)
  train_test_switch : True for train ,False for test
  
  The overall use of this class is :
  >>TestObj = TSNdataset(......)                    #Creating an object
  >>TestObj[Index]                                  #Index : is the index for a video in the dataset (0:13321)
  
  previous line gives a tuple (List of the picked frames,The label for this video)
  
  
  '''
  def __init__(self, root_path, file_list,num_segments=3, new_length=1, modality='RGB',
                 image_prefix='frame{:06d}.jpg', transform=None,test_mode=False ,train_test_switch=True):
    
    self.root_path = root_path
    self.file_list = file_list
    self.num_segments = num_segments
    self.new_length = new_length
    self.modality = modality
    self.image_prefix = image_prefix
    self.transform = transform
    self.test_mode = test_mode
    
    if self.modality == 'RGBDiff' :
      new_length += 1                                           
    
    #List of Objects, Everyone of them corresponding to a video
    self.VidInfoList = [VidInfo(x.strip().split('  '))  for x in self.file_list]               
  
  
  def Load_Img (self,directory,idx):
    '''
    directory : Video path
    idx : Frames suffix (Frame number eg. 000013)
    '''
    return [Image.open(os.path.join(directory,self.image_prefix.format(idx))).convert('RGB')]
  
  
  
  def Train_Sample_indices(self,vid_info):
    '''
    vid_info : Object has info about one video
    This function determine the indices for the chosen frames
    '''
    if vid_info.num_frames < self.num_segments:
      return np.array(range(vid_info.num_frames))+1                                             #Return all frames inside short video
    
    FPSeg = vid_info.num_frames // self.num_segments                                            #Frames per segment
    offest = [x*FPSeg for x in range(num_segments)]                                             #The first index for every segment 
    frame_indices = list(randint(FPSeg,size=self.num_segments))                                 #Chosen frames form every segment
    sample_indices = [sum(i) for i in zip(frame_indices,offest)]                                #elment-wise sumtion of offest and smaple_indices
    return np.array(sample_indices)+1                                                           #Frames indices starts from 000001
  
  
  
  def Val_Sample_indices(self,vid_info):
    '''
    vid_info : Object has info about one video
    This function determine the indices for the chosen frames
    '''
    if vid_info.num_frames < self.num_segments:
      return np.array(range(vid_info.num_frames))+1
    
    FPSeg = vid_info.num_frames / float(self.num_segments)
    sample_indices = [int(FPSeg*( x + 1/2.0 )) for x in range(self.num_segments)]               #Get the middle frame for every segment
    return np.array(sample_indices)+1
  
  
  
  def Test_Sample_indices(self,vid_info):
    '''
    vid_info : Object has info about one video
    This function determine the indices for the chosen frames
    '''    
    if vid_info.num_frames < self.num_segments:
      return np.array(range(vid_info.num_frames))+1
    
    FPSeg = vid_info.num_frames / float(self.num_segments)
    sample_indices = [int(FPSeg*( x + 1/2.0 )) for x in range(self.num_segments)]
    return np.array(sample_indices)+1
    
    
    
  def Vid2Frames(self,vid_info,indices):
    '''
    vid_info : Object has info about one video
    indices  : Indices for the chosen frames
    '''
    images = []
    for idx in indices:                                                         #For every frame in the chosen frames
      Img = self.Load_Img (vid_info.path,idx)                                   #Load the frame
      images.extend(Img)                                                        #Stack it with the others
      
    processed_frames = self.transform(images)                                   #Applay transform stuff on the chosen frames
    return processed_frames, vid_info.label                                       
  
  
  
  def __getitem__(self,idx):
    '''
    idx : Index for a video into the dataset
    The main perpuse of this function is to interact 'nicer' with the object of the class
    >>TestObj[3] 
    This line output a tuple (List of chosen frames,label) for the 3rd video in the dataset
    '''
    vid_info = self.VidInfoList[idx]
    if test_mode :
      indices = self.Test_Sample_indices(vid_info)
    else:
      indices = self.Trian_Sample_indices(vid_info) if train_test_swtich else self.Val_Sample_indices(vid_info)  
    return self.Vid2Frames(vid_info,indices)
    
  
  
  